<a href="https://colab.research.google.com/github/tsai-jiewen/ninja/blob/main/Lab7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 7

- 蔡介文 109152512
- 2022/11/08

請在梯度下降法、牛頓法、BFGS法加入backtracking（s_0 = 1, \gamma = .5, c = .01），並比較其在P=10, 100, 1000之表現，包括迭代次數、總時間、以及每次迭代所需之時間

In [ ]:
import jax.numpy as jnp
import jax
from jax import grad, hessian, random
import time

import matplotlib.pyplot as plt


jax.config.update("jax_enable_x64", True)

## 1.資料

In [ ]:
# testing data

def gen_logistic_data(n_cases,beta,corr = 0,key = None):
    if key is None:
        key = jax.random.PRNGKey(48)
        n_features = len(beta) -1
        mu = jnp.zeros(n_features)
        sigma = (corr * jnp.ones((n_features, n_features)) + (1 - corr) * jnp.eye(n_features))
        key, subkey = jax.random.split(key)
        x0 = jax.random.multivariate_normal(key = key, mean = mu,cov = sigma,shape = (n_cases, ))
        eta = beta[0] + x0 @ beta[1:]
        pi = 1 / (1 + jnp.exp(-eta))
        key, subkey = jax.random.split(key)
        y = 1. * jax.random.bernoulli(key = key,p = pi)
    return y, x0

def get_yxb0(n_cases = 10000, n_beta=10, corr = .3):
    """
    Return:
        y:
        x:
        b0:
    """
    key1 = jax.random.PRNGKey(1234)
    true_beta = jax.random.uniform(key=key1, shape=(1,n_beta) )[0]
    y, x0 = gen_logistic_data(n_cases = n_cases,beta = true_beta,corr = corr,key = None)
    x = jnp.hstack((jnp.ones((x0.shape[0],1)), x0))
    b0 = random.normal( key1, shape = (1,n_beta))[0]
    # b0 = jnp.zeros(n_beta)
    return y, x, b0

In [ ]:
y, x, b0 = get_yxb0(n_beta=10)

In [ ]:
b0

DeviceArray([ 0.46265712, -0.2214463 , -1.51707684, -0.1593065 ,
              0.11869523, -0.85134133, -0.22682583,  0.23867022,
             -0.32325433, -0.08618864], dtype=float64)

## 2.目標函數

In [ ]:
def y_hat(b, x):
    return 1 / (1 + jnp.exp( - x @ b ) )

def loglik(b, x, y):
    loss = y * jnp.log( y_hat(b,x)) + (1 - y) * jnp.log((1 - y_hat(b,x)) )
    return - jnp.mean( loss )



In [ ]:
dloglik = grad(loglik, argnums=0)
d2loglik = hessian(loglik, argnums=0)

In [ ]:
# b1 = jnp.zeros(n_beta)
d2loglik(b0, x, y)

DeviceArray([[ 0.13826517,  0.00704425,  0.01595939,  0.00878399,
               0.00383845,  0.01383925,  0.00667438,  0.00574951,
               0.00891384,  0.00811783],
             [ 0.00704425,  0.12098645,  0.00702521,  0.02163728,
               0.02503225,  0.01607399,  0.02329883,  0.03039352,
               0.02098852,  0.02439288],
             [ 0.01595939,  0.00702521,  0.07125191,  0.00767809,
               0.01155596, -0.00881163,  0.00498604,  0.01521263,
               0.00144692,  0.00860969],
             [ 0.00878399,  0.02163728,  0.00767809,  0.11973625,
               0.02720598,  0.01733415,  0.02398808,  0.02946857,
               0.01902268,  0.02537211],
             [ 0.00383845,  0.02503225,  0.01155596,  0.02720598,
               0.12799636,  0.02146672,  0.0280943 ,  0.03285649,
               0.02409468,  0.0287764 ],
             [ 0.01383925,  0.01607399, -0.00881163,  0.01733415,
               0.02146672,  0.09839812,  0.01587616,  0.02413315,
   

## 3.方法

### 3.1 LogisticRegression

In [ ]:
from sklearn import linear_model

log_reg = linear_model.LogisticRegression( penalty='none', tol=1e-5, fit_intercept=True ).fit(x, y)

# b0 as the fitting beta vector
b1 = jnp.array( log_reg.coef_ )
b1
#log_reg.intercept_

DeviceArray([[ 0.36782415,  0.48829364,  0.34299356,  0.43387493,
               0.7868046 ,  1.1468894 ,  0.17357975,  0.77271533,
               0.01443904,  0.66009202,  0.20720668,  0.82879324,
               1.11163723,  0.25088968,  0.67566425,  0.50757679,
               0.48781801,  0.18242385,  1.20824879,  1.24106539,
               0.18724431,  0.74628291,  0.48638817,  0.88614284,
               1.00975148,  0.86748026,  0.41864433,  0.27447976,
               1.0168899 ,  1.17121037,  1.09393871,  0.81627559,
               0.0406182 ,  0.44854451, -0.26983948,  0.52908733,
               0.12748756,  1.4337683 ,  1.20370922,  0.40287978,
               0.73981512,  0.40658254,  0.61064665,  0.20334093,
              -0.10085614,  1.30069352,  0.80887636,  1.05294395,
               0.42243243,  0.84957519,  1.23759721,  0.55730226,
               1.01253428,  0.45829379, -0.26535905,  0.30833496,
               0.63033466,  0.69512134,  0.19072043,  0.72718179,
          

### 3.2 梯度下降

In [ ]:
def descent(f=loglik, b=b0, s = 1., epochs = 1000, tol = 1e-5, c=1e-2, gamma=.5):
    """Gradient descent with backtracking line search.
    Args:
        f: loglik function
        b: parameters
        s: the maximum step size
        epochs: steps.
        tol: tolerence
        c: the first Wolfe condition parameter
        gamma: the reduction factor
    Returns:
        epoch+1: total epochs
        ll: LogLikelihood
        b: Beta parameters
        s: Final Step Size
        dll: Final LogLik Change
    """
    for epoch in range(epochs):
        df = dloglik(b, x, y)
        d = -df
        while f((b + s*d),x,y) > f(b,x,y) + c*s*( d.T @ df ):
            s *= gamma
        b_tmp = b
        b += s*d

        ll = f(b,x,y)
        dll = f(b_tmp,x,y) - f(b,x,y)

        if (jnp.abs(d).max() < tol):
            break

    if (epoch==epochs-1):
        print('!!! Warning: DESCENT did not converge.')

    return epoch+1, ll, b, s, dll




### 3.3 牛頓法

In [ ]:
def newton(f=loglik, b=b0, s = 1., epochs = 1000, tol = 1e-5, c=1e-2, gamma=.5):
    """Newton method with backtracking line search.
    Args:
        f: loglik function
        b: parameters
        s: the maximum step size
        epochs: steps.
        tol: tolerence
        c: the first Wolfe condition parameter
        gamma: the reduction factor
    Returns:
        epoch+1: total epochs
        ll: LogLikelihood
        b: Beta parameters
        s: Final Step Size
        dll: Final LogLik Change
    """
    for epoch in range(epochs):
        df = dloglik(b, x, y)
        d2f = d2loglik(b, x, y)
        d = - (jnp.linalg.inv(d2f) @ df)

        while f( (b + s*d), x,y ) > f(b,x,y) + c*s*( d.T @ df ):
            s *= gamma
        b_tmp = b
        b += s*d

        ll = f(b,x,y)
        dll = f(b_tmp,x,y) - f(b,x,y)
        if (jnp.abs(d).max() < tol):
            break

    if (epoch==epochs-1):
        print('!!! Warning: NEWTON did not converge.')

    return epoch+1, ll, b, s, dll



### 3.4 BFGS

In [ ]:
def h_itr(b, d, h, idt, f, s, c, gamma):
    """The iteration part of BFGS.
    Args:
        b: parameters
        d: a descent direction
        h: approximate Hessian
        idt: initial approximate Hessian
        f: loglik function
        s: the maximum step size
        c: the first Wolfe condition parameter
        gamma: the reduction factor
    Returns:
        h: updating approximate Hessian
        b: updating beta
        d: updating descent direction
        s: updating s
    """
    b_new = b + s * (h @ d)
    df = dloglik(b_new, x, y)
    dll = loglik(b, x, y) - loglik(b_new, x, y)
    d_new = - df

    while f((b + s*d_new), x,y) > f(b, x,y) + c*s*( d_new.T @ df ):
        s *= gamma

    u = b_new - b
    v = d - d_new
    rho = 1 / jnp.inner(u, v)
    uv = jnp.outer(u, v)
    uu = jnp.outer(u, u)
    h = (idt - rho * uv) @ h @ (idt - rho * uv) + rho * uu
    b = b_new
    d = d_new

    return h, b, d, s, dll


def bfgs(f=loglik, b=b0, s = 1., epochs = 1000, tol = 1e-5, c=1e-2, gamma=.5):
    """BFGS with backtracking line search.
    Args:
        f: loglik function
        b: parameters
        s: the maximum step size
        epochs: steps.
        tol: tolerence
        c: the first Wolfe condition parameter
        gamma: the reduction factor
    Returns:
        epoch+1: total epochs
        ll: LogLikelihood
        b: Beta parameters
        s: Final Step Size
        dll: Final LogLik Change
    """
    d = - dloglik(b, x, y)
    h = jnp.eye( len(b) )
    idt = jnp.eye( len(b) )

    for epoch in range(epochs):
        ll = f(b, x,y)
        h, b, d, s, dll = h_itr(b, d, h, idt, f, s, c, gamma)

        if (jnp.abs(d).max() < tol):
            break

    if (epoch==epochs-1):
        print('!!! Warning: BFGS did not converge.')

    return epoch+1, ll, b, s, dll




## 4.比較

sklearn, 梯度下降, 牛頓法, BFGS

### 10 Parameters

In [ ]:
"""
Prepare: y, x, b0, dloglik, d2loglik
"""
y, x, b10 = get_yxb0(n_beta=10)
dloglik = grad(loglik, argnums=0)
d2loglik = hessian(loglik, argnums=0)


# --------------------------------------- #

str_d10 = time.time()
ep_d10, ll_d10, b_d10, s_d10, dll_d10 = descent(f=loglik, b=b10, c=1e-4)
end_d10 = time.time()

# Report
# print("================================")
print('             DESCENT')
print("-------------------------------")
print('1) Epochs:', ep_d10)
print('2) Time: {:.3f} (s)'.format(end_d10 - str_d10))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_d10 - str_d10)/ep_d10))
print('4) LogLik:' ,round(ll_d10,3))
print('5) Final ΔLogLik: ' , dll_d10)
print('6) Final StepSize:' ,s_d10)
print('7) Beta:  ', round(b_d10,3))
print("================================")

# --------------------------------------- #

str_n10 = time.time()
ep_n10, ll_n10, b_n10, s_n10, dll_n10 = newton(f=loglik, b=b10, c=1e-4)
end_n10 = time.time()

# Report
# print("================================")
print('             NEWTON')
print("-------------------------------")
print('1) Epochs:', ep_n10)
print('2) Time: {:.3f} (s)'.format(end_n10 - str_n10))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_n10 - str_n10)/ep_n10))
print('4) LogLik:' ,round(ll_n10,3))
print('5) Final ΔLogLik: ' , dll_n10)
print('6) Final StepSize:' ,s_n10)
print('7) Beta:  ', round(b_n10,3))
print("================================")


# --------------------------------------- #

str_b10 = time.time()
ep_b10, ll_b10, b_b10, s_b10, dll_b10 = bfgs(f=loglik, b=b10, c=1e-4)
end_b10 = time.time()

# Report
# print("================================")
print('             BFGS')
print("-------------------------------")
print('1) Epochs:', ep_b10)
print('2) Time: {:.3f} (s)'.format(end_b10 - str_b10))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_b10 - str_b10)/ep_b10))
print('4) LogLik:' ,round(ll_b10,3))
print('5) Final ΔLogLik: ' , dll_b10)
print('6) Final StepSize:' ,s_b10)
print('7) Beta:  ', round(b_b10,3))
print("================================")

             DESCENT
-------------------------------
1) Epochs: 91
2) Time: 7.900 (s)
3) Time/Epoch: 0.087 (s/Epoch)
4) LogLik: 0.433
5) Final ΔLogLik:  3.064821174625365e-10
6) Final StepSize: 1.0
7) Beta:   [0.67  0.388 0.055 0.417 0.545 0.226 0.433 0.592 0.354 0.423]
!!! Warning: NEWTON did not converge.
             NEWTON
-------------------------------
1) Epochs: 1000
2) Time: 59.266 (s)
3) Time/Epoch: 0.059 (s/Epoch)
4) LogLik: nan
5) Final ΔLogLik:  nan
6) Final StepSize: 1.0
7) Beta:   [nan nan nan nan nan nan nan nan nan nan]


KeyboardInterrupt: ignored

### 100 Parameters

In [ ]:
"""
Prepare: y, x, b0, dloglik, d2loglik
"""
y, x, b100 = get_yxb0(n_beta=100)
dloglik = grad(loglik, argnums=0)
d2loglik = hessian(loglik, argnums=0)

str_d100 = time.time()
ep_d100, ll_d100, b_d100, s_d100, dll_d100 = descent(f=loglik, b=b100, s=.001)
end_d100 = time.time()

# Report
# print("================================")
print('             DESCENT')
print("-------------------------------")
print('1) Epochs:', ep_d100)
print('2) Time: {:.3f} (s)'.format(end_d100 - str_d100))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_d100 - str_d100)/ep_d100))
print('4) LogLik:' ,round(ll_d100,3))
print('5) Final ΔLogLik: ' , dll_d100)
print('6) Final StepSize:' ,s_d100)
print('7) Beta:  ', round(b_d100,3))
print("================================")

!!! Warning: DESCENT did not converge.
             DESCENT
-------------------------------
1) Epochs: 1000
2) Time: 27.232 (s)
3) Time/Epoch: 0.027 (s/Epoch)
4) LogLik: nan
5) Final ΔLogLik:  nan
6) Final StepSize: 0.001
7) Beta:   [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]


In [ ]:
"""
Prepare: y, x, b0, dloglik, d2loglik
"""
y, x, b100 = get_yxb0(n_beta=100)
dloglik = grad(loglik, argnums=0)
d2loglik = hessian(loglik, argnums=0)



# --------------------------------------- #

str_d100 = time.time()
ep_d100, ll_d100, b_d100, s_d100, dll_d100 = descent(f=loglik, b=b100, s=2)
end_d100 = time.time()

# Report
# print("================================")
print('             DESCENT')
print("-------------------------------")
print('1) Epochs:', ep_d100)
print('2) Time: {:.3f} (s)'.format(end_d100 - str_d100))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_d100 - str_d100)/ep_d100))
print('4) LogLik:' ,round(ll_d100,3))
print('5) Final ΔLogLik: ' , dll_d100)
print('6) Final StepSize:' ,s_d100)
print('7) Beta:  ', round(b_d100,3))
print("================================")

# --------------------------------------- #

str_n100 = time.time()
ep_n100, ll_n100, b_n100, s_n100, dll_n100 = newton(f=loglik, b=b100, c=1e-5)
end_n100 = time.time()

# Report
# print("================================")
print('             NEWTON')
print("-------------------------------")
print('1) Epochs:', ep_n100)
print('2) Time: {:.3f} (s)'.format(end_n100 - str_n100))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_n100 - str_n100)/ep_n100))
print('4) LogLik:' ,round(ll_n100,3))
print('5) Final ΔLogLik: ' , dll_n100)
print('6) Final StepSize:' ,s_n100)
print('7) Beta:  ', round(b_n100,3))
print("================================")


# --------------------------------------- #

str_b100 = time.time()
ep_b100, ll_b100, b_b100, s_b100, dll_b100 = bfgs(f=loglik, b=b100, c=1e-5)
end_b100 = time.time()

# Report
# print("================================")
print('             BFGS')
print("-------------------------------")
print('1) Epochs:', ep_b100)
print('2) Time: {:.3f} (s)'.format(end_b100 - str_b100))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_b100 - str_b100)/ep_b100))
print('4) LogLik:' ,round(ll_b100,3))
print('5) Final ΔLogLik: ' , dll_b100)
print('6) Final StepSize:' ,s_b100)
print('7) Beta:  ', round(b_b100,3))
print("================================")

!!! Warning: DESCENT did not converge.
             DESCENT
-------------------------------
1) Epochs: 1000
2) Time: 25.417 (s)
3) Time/Epoch: 0.025 (s/Epoch)
4) LogLik: nan
5) Final ΔLogLik:  nan
6) Final StepSize: 2
7) Beta:   [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]


KeyboardInterrupt: ignored

### 1000 Parameters

In [ ]:
"""
Prepare: y, x, b0, dloglik, d2loglik
"""
y, x, b1000 = get_yxb0(n_beta=1000)
dloglik = grad(loglik, argnums=0)
d2loglik = hessian(loglik, argnums=0)
b1000_ini = dloglik(b1000, x, y)


# --------------------------------------- #

str_d1000 = time.time()
ep_d1000, ll_d1000, b_d1000, s_d1000, dll_d1000 = descent(f=loglik, b=b1000_ini)
end_d1000 = time.time()

# Report
# print("================================")
print('             DESCENT')
print("-------------------------------")
print('1) Epochs:', ep_d1000)
print('2) Time: {:.3f} (s)'.format(end_d1000 - str_d1000))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_d1000 - str_d1000)/ep_d1000))
print('4) LogLik:' ,round(ll_d1000,3))
print('5) Final ΔLogLik: ' , dll_d1000)
print('6) Final StepSize:' ,s_d1000)
print('7) Beta:  ', round(b_d1000[:10],3))
print("================================")

# --------------------------------------- #

str_n1000 = time.time()
ep_n1000, ll_n1000, b_n1000, s_n1000, dll_n1000 = newton(f=loglik, b=b1000_ini)
end_n1000 = time.time()

# Report
# print("================================")
print('             NEWTON')
print("-------------------------------")
print('1) Epochs:', ep_n1000)
print('2) Time: {:.3f} (s)'.format(end_n1000 - str_n1000))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_n1000 - str_n1000)/ep_n1000))
print('4) LogLik:' ,round(ll_n1000,3))
print('5) Final ΔLogLik: ' , dll_n1000)
print('6) Final StepSize:' ,s_n1000)
print('7) Beta:  ', round(b_n1000[:10],3))
print("================================")


# --------------------------------------- #

str_b1000 = time.time()
ep_b1000, ll_b1000, b_b1000, s_b1000, dll_b1000 = bfgs(f=loglik, b=b1000_ini)
end_b1000 = time.time()

# Report
# print("================================")
print('             BFGS')
print("-------------------------------")
print('1) Epochs:', ep_b1000)
print('2) Time: {:.3f} (s)'.format(end_b1000 - str_b1000))
print('3) Time/Epoch: {:.3f} (s/Epoch)'.format((end_b1000 - str_b1000)/ep_b1000))
print('4) LogLik:' ,round(ll_b1000,3))
print('5) Final ΔLogLik: ' , dll_b1000)
print('6) Final StepSize:' ,s_b1000)
print('7) Beta:  ', round(b_b1000[:10],3))
print("================================")

!!! Warning: DESCENT did not converge.
             DESCENT
-------------------------------
1) Epochs: 1000
2) Time: 29.837 (s)
3) Time/Epoch: 0.030 (s/Epoch)
4) LogLik: nan
5) Final ΔLogLik:  nan
6) Final StepSize: 1.0
7) Beta:   [nan nan nan nan nan nan nan nan nan nan]
!!! Warning: NEWTON did not converge.
             NEWTON
-------------------------------
1) Epochs: 1000
2) Time: 434.779 (s)
3) Time/Epoch: 0.435 (s/Epoch)
4) LogLik: nan
5) Final ΔLogLik:  nan
6) Final StepSize: 1.0
7) Beta:   [nan nan nan nan nan nan nan nan nan nan]
!!! Warning: BFGS did not converge.
             BFGS
-------------------------------
1) Epochs: 1000
2) Time: 45.997 (s)
3) Time/Epoch: 0.046 (s/Epoch)
4) LogLik: nan
5) Final ΔLogLik:  nan
6) Final StepSize: 1.0
7) Beta:   [nan nan nan nan nan nan nan nan nan nan]
